In [ ]:
import pandas as pd

from scripts.analysis import data

repos = data.get_fake_star_repos_all()
repos_dagster = pd.read_csv("data/dagster_test/fake_stars_complex_repos.csv")
stars_dagster = pd.read_csv("data/dagster_test/fake_stars_complex_users.csv")

repos_dagster = repos_dagster[(repos_dagster.n_fake_stars >= 50)]
repo_ids_dagster = set(repos_dagster.repo_id)

print(
    len(repo_ids_dagster),
    len(set(repos.repo_id) & repo_ids_dagster),
    repo_ids_dagster - set(repos.repo_id),
)
perfs = []
for repo_id in set(repos.repo_id) & repo_ids_dagster:
    repo_names = set(repos[repos.repo_id == repo_id].repo_name)
    fakes_copycatch, fakes_dagster = set(), set()
    for repo_name in repo_names:
        df = data.get_stars_from_repo(repo_name)
        if df is None:
            continue
        fakes_copycatch |= set(df[df.low_activity | df.clustered].actor)
        fakes_dagster |= set(
            stars_dagster[
                (stars_dagster.repo_name == repo_name)
                & (stars_dagster.fake_acct != "unknown")
            ].actor
        )
    perfs.append(
        {
            "repo_names": repo_names,
            "n_fakes_copycatch": len(fakes_copycatch),
            "n_fakes_dagster": len(fakes_dagster),
            "n_overlap": len(fakes_copycatch & fakes_dagster),
        }
    )
perfs = pd.DataFrame(perfs)
print("Precision:", perfs.n_overlap.sum() / perfs.n_fakes_copycatch.sum())
print("Recall:", perfs.n_overlap.sum() / perfs.n_fakes_dagster.sum())
perfs